# Visualizing Learning Curves in Jupyter Notebook

This notebook demonstrates a visualization utility of Optuna.
After optimizing the hyperparameter of neural networks, `plot_intermediate_values()` plots learning curves of running, completed, and pruned trials in a study.

**Note:** You need to set up the pruning feature to utilize `plot_intermediate_values()`; otherwise, no data appears in the plotted figure. Please see also [the pruning tutorial](https://optuna.readthedocs.io/en/stable/tutorial/pruning.html) for more details.

In [1]:
# モジュールimport
import os, sys
sys.path.append(r'C:\Users\shingo\Git\optuna')
%matplotlib inline

## Setting up MNIST Dataset

In [2]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

mnist = fetch_openml('mnist_784', version=1)
classes = list(set(mnist.target))
x_train, x_test, y_train, y_test = train_test_split(mnist.data, mnist.target)

## Defining Objective Function

In [3]:
from sklearn.neural_network import MLPClassifier

def objective(trial):

    layers = []
    n_layers = trial.suggest_int('n_layers', 1, 4)
    for i in range(n_layers):
        layers.append(trial.suggest_int('n_units_l{}'.format(i), 1, 128))

    clf = MLPClassifier(hidden_layer_sizes=tuple(layers))

    for step in range(100):
        clf.partial_fit(x_train, y_train, classes=classes)

        intermediate_value = 1.0 - clf.score(x_test, y_test)  # Report intermediate objective value.
        trial.report(intermediate_value, step)

        if trial.should_prune(step):
            raise optuna.structs.TrialPruned()  # Handle pruning based on the intermediate value.

    return 1.0 - clf.score(x_test, y_test)

## Running Optimization

In [4]:
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)  # This verbosity change is just to simplify the notebook output.

study = optuna.create_study(pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=25)

Using TensorFlow backend.


## Plotting Learning Curves of Trials

In [5]:
from optuna.visualization import plot_intermediate_values

plot_intermediate_values(study)

**Note:** The plotted figure is not rendered on GitHub due to a compatibility issue between Plotly and GitHub. Please try running the notebook on your environment, and you can see `plot_intermediate_values()` plot a figure as follows.

![plot_intermediate_values.png](plot_intermediate_values.png)